In [2]:
import pandas as pd
import numpy as np
import os


In [18]:
# step 1: delete the duplicated lines

f = open('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1.csv')
lines = f.readlines()
set_lines = set(lines[1:])
#print (len(lines), len(set(lines)))
new_list = sorted(list(set_lines))
f_new = open('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', 'w')
f_new.write(lines[0])
for i in range(len(new_list)):
    f_new.write(new_list[i])
f_new.close()


In [93]:
# add the MCL id to Jasper file
f = open('/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data_from_NFS0421/MCL/LabelFile/0330/VUMC/MRN_in_Jasper_dict.txt')
lines = f.readlines()[1:]
lines = [re.split('[ \t]', i.strip()) for i in lines]
MRN_MCL = {}
for i in range(len(lines)):
    if '0' <= lines[i][0][0] and '9'>= lines[i][0][0]:
        assert len(lines[i]) == 2
        MRN_MCL[lines[i][0]] = lines[i][1]
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
mcl_list = []
MRN_MCL['nan'] = 'nan'
for i, item in df.iterrows():
    mrn = str(item['MRN']).replace('.0', '')
    if mrn[0] == '0':
        try:
            mrn = str(int(mrn))
        except:
            mrn = mrn
    if mrn not in MRN_MCL.keys():
        mcl_list.append('')
        print (mrn)
    else:
        mcl_list.append(MRN_MCL[mrn])
df['MCL_ID'] = mcl_list
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

dup_of_13797
dup_of_13797
va0000000
test
VA De iden03
.
.
dup_of_8116_1
..
..
VADEIDEN2
VA De iden04
VADEIDENT
VADEIDENT
VADEIDEN1
dup_of_8116_2
Dup_7007
dup_of_7727
dup_of_7785
dup_of_7818
dup_of_7047
dup_of_8203
dup_of_9366
034140467+
Dupl_of 8921
dup_of_9403
dup_of_9560
dup_of_9618


In [94]:
# compute with image 

df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0330/VUMC/Riqiang_3_30_2020_16_52_58.csv')
subj_list = df['Subject'].tolist()
subj_list = [re.split('[-_.]', str(i))[0] for i in subj_list]
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')

withimg_0330 = []
withimg_sub = []

for i, item in df.iterrows():
    if str(item['MCL_ID']).replace('.0', '') in subj_list:
        withimg_0330.append(1)
        withimg_sub.append(item['MCL_ID'])
    else:
        withimg_0330.append(0)
print (sum(withimg_0330), len(set(withimg_sub)))
df['withimg_0330'] = withimg_0330
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

1867 996


In [95]:
# compute how many subject with label in VUMC and VA
#
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0330/VUMC/Riqiang_3_30_2020_16_52_58.csv')

subj_list = df['Subject'].tolist()
subj_list = [re.split('[-_.]', str(i))[0] for i in subj_list]
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
img_pos = []
img_neg = []
for i, item in df.iterrows():
    if item['withimg_0330'] == 1 and item['cancer_bengin'] == 1:
        img_pos.append(item['MCL_ID'])
    elif item['withimg_0330'] == 1 and item['cancer_bengin'] == 0:
        img_neg.append(item['MCL_ID'])
img_pos = [str(i).replace('.0', '') for i in img_pos]
img_neg = [str(i).replace('.0', '') for i in img_neg]
print (len(set(img_pos)), len(set(img_neg)), len(set(img_neg) & set(img_pos)))
print (len(set(img_pos)) + len(set(img_neg)) - len(set(img_neg) & set(img_pos)))

654 372 72
954


In [96]:
# add the cancer_bengin

df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
cancer_set = ['Sarcoma', 'Adenoid Cystic Carcinoma', 'Squamous Cell Carcinoma', 'Squamous Metaplasia', 'Adenosquamous Carcinoma', 'Small Cell Carcinoma', 'Adenocarcinoma', 'Large Cell Neuroendocrine', 'Large Cell Carcinoma', 'Bronchioalveolar Carcinoma', 'Carcinoid', 'Moderate Dysplasia', 'Atypical Carcinoid', 'Non Small Cell (NSCLC)', 'Carcinoma In Situ']
nocancer_set = ['Negative for Malignant Cells', 'Normal', 'Negative for Dysplasia and Metaplasia', 'Granuloma ', 'Granuloma' , 'Mild Dysplasia']
null_set = ['nan', 'Other', 'No Diagnosis', 'Unsatisfactory']
cancer_bengin = []
for i, item in df.iterrows():
    
    if item['Histologic Type'] !=  item['Histologic Type'] or item['Histologic Type'] in null_set:
        cancer_bengin.append('')
    elif item['Histologic Type'] in cancer_set:
        cancer_bengin.append(1)
    elif item['Histologic Type'] in nocancer_set:
        cancer_bengin.append(0)
df['cancer_bengin'] = cancer_bengin
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

In [97]:
# add the protocol name to Report_part1_nodup.csv
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0330/VUMC/Report_protocol.csv')
pid_in0398 = {}
pid_in1730 = {}
pid_in0136 = {}
pid_in1078 = {}
for i, item in df.iterrows():
    if item['Patient ID'] not in pid_in0398.keys():
        pid_in0398[item['Patient ID']] = 0
    
    if item['Patient ID'] not in pid_in1730.keys():
        pid_in1730[item['Patient ID']] = 0
        
    if item['Patient ID'] not in pid_in0136.keys():
        pid_in0136[item['Patient ID']] = 0
    
    if item['Patient ID'] not in pid_in1078.keys():
        pid_in1078[item['Patient ID']] = 0
        
    pid_in0398[item['Patient ID']]  = max(item['in0398'], pid_in0398[item['Patient ID']])
    pid_in1730[item['Patient ID']]  = max(item['in1730'], pid_in1730[item['Patient ID']])
    pid_in0136[item['Patient ID']]  = max(item['in0136'], pid_in0136[item['Patient ID']])
    pid_in1078[item['Patient ID']]  = max(item['in1078'], pid_in1078[item['Patient ID']])
    
    
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
in0398, in1730, in0136, in1078 = [], [], [],[]

for i, item in df.iterrows():
    if item['Patient ID'] not in pid_in0398:
        in0398.append(0)
    else:
        in0398.append(pid_in0398[item['Patient ID']])
        
    if item['Patient ID'] not in pid_in1730:
        in1730.append(0)
    else:
        in1730.append(pid_in1730[item['Patient ID']])
        
    if item['Patient ID'] not in pid_in0136:
        in0136.append(0)
    else:
        in0136.append(pid_in0136[item['Patient ID']])
        
    if item['Patient ID'] not in pid_in1078:
        in1078.append(0)
    else:    
        in1078.append(pid_in1078[item['Patient ID']])
    
df['in0398'] = in0398
df['in1730'] = in1730
df['in0136'] = in0136
df['in1078'] = in1078

df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

In [99]:
df1 = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0330/VUMC/Report_protocol1.csv')
df2 = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0330/VUMC/Report_protocol2.csv')
s0398_set, s1730_set, s0136_set, s1078_set = set(), set(), set(), set()
for i, item in df1.iterrows():
    if '0398' in str(item['Correlative Protocol']):
        s0398_set.add(item['Patient ID'])
    if '1730' in str(item['Correlative Protocol']):
        s1730_set.add(item['Patient ID'])
    if '0136' in str(item['Correlative Protocol']):
        s0136_set.add(item['Patient ID'])
    if '1078' in str(item['Correlative Protocol']):
        s1078_set.add(item['Patient ID'])
        
for i, item in df2.iterrows():
    if '0398' in str(item['Protocol Name']):
        s0398_set.add(item['Patient ID'])
    if '1730' in str(item['Protocol Name']):
        s1730_set.add(item['Patient ID'])
    if '0136' in str(item['Protocol Name']):
        s0136_set.add(item['Patient ID'])
    if '1078' in str(item['Protocol Name']):
        s1078_set.add(item['Patient ID'])

In [100]:
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
in0398, in1730, in0136, in1078 = [], [], [],[]

for i, item in df.iterrows():
    if item['Patient ID'] not in s0398_set:
        in0398.append(0)
    else:
        in0398.append(1)
        
    if item['Patient ID'] not in s1730_set:
        in1730.append(0)
    else:
        in1730.append(1)
        
    if item['Patient ID'] not in s0136_set:
        in0136.append(0)
    else:
        in0136.append(1)
        
    if item['Patient ID'] not in s1078_set:
        in1078.append(0)
    else:    
        in1078.append(1)
    
df['new_in0398'] = in0398
df['new_in1730'] = in1730
df['new_in0136'] = in0136
df['new_in1078'] = in1078

df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

In [103]:
# for statisitic
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
#df = df.query('cancer_bengin == cancer_bengin')
df = df.loc[df['withimg_0330'] == 1]
df_0 = df.loc[(df['new_in0136'] == 1) | (df['new_in0398'] == 1)]
MCL_list = df_0['MRN'].tolist()
MCL_list = [str(i).replace('.0', '') for i in MCL_list]
print (len(set(MCL_list)))

741


In [118]:
# got the spicu and upper lobe 
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
spicul = []
for i, item in df.iterrows():
    if 'spicu' in str(item['Chest CT Notes']) or 'spicu' in str(item['PET Notes']):
        spicul.append(1)
    else:
        spicul.append(0)

allset = (set(df['Chest CT Location'].tolist()))
Upper_set = ['LUL/RLL',  'LUL&RLL','LUL','RUL & LUL',  'LUL & RUL','LUL & LLL','RUL, LUL', 'RML & LUL',  
' LUL', 'RUL ','RLL & LUL',  'LUL ', 'RLL & RUL',  'RUL & RLL',  'RUL',  'RUL(hilar)']
upper_list = []
for i, item in df.iterrows():
    if str(item['Chest CT Location']) in Upper_set:
        upper_list.append(1)
    else:
        upper_list.append(0)
df['spicul'] = spicul
df['upperlobe'] = upper_list
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)
print (sum(spicul), sum(upper_list), len(df))

403 2190 10628


In [108]:
# normalize the somking and nodule size

df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
nodule_size, smok = [], []
#print (set(df['CIGARETTE_Q1'].tolist()))
prior_cancer = []
for i, item in df.iterrows():
    if item['Prior Cancer'] != item['Prior Cancer']:
        prior_cancer.append('')
    elif item['Prior Cancer'] == 'Yes':
        prior_cancer.append(1)
    elif item['Prior Cancer'] == 'No':
        prior_cancer.append(0)
    if item['CIGARETTE_Q1'] != item['CIGARETTE_Q1'] or item['CIGARETTE_Q1'] == 'Unknown':
        smok.append('')
    elif item['CIGARETTE_Q1'] == 'Never smoked':
        smok.append(0)
    elif item['CIGARETTE_Q1'] in ['Current smoker', 'Ex-smoker']:
        smok.append(1)
    if item['Chest CT Size'] != item['Chest CT Size']:
        nodule_size.append('')
    else:
        if item['Chest CT Size'] > 15:
            print (i, item['Patient ID'])
        nodule_size.append(item['Chest CT Size'] * 10)
    
df['nodule_size'] = nodule_size
df['smok'] = smok
df['prior_cancer'] = prior_cancer
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

758 11578
759 11578
8781 6829


In [122]:
from datetime import datetime
import re

def convert_to_time1(string):
    month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    str_vec = re.split('[ ,]', string)
    str_vec = [i for i in str_vec if len(i) > 0]
    #print (str_vec)
    assert len(str_vec) == 3
    return datetime(int(str_vec[2]), month_dict[str_vec[0]], int(str_vec[1]))

def convert_to_time2(string):
    str_vec = re.split('[/]', string)
    
    assert len(str_vec) == 3
    int_vec = [int(i) for i in str_vec]
    
    assert int_vec[0] < 13 and int_vec[1] < 32 and int_vec[2] < 100
#     except:
#         print (string)
    if int_vec[2] < 21:
        int_vec[2] += 2000
    else:
        int_vec[2] += 1900
    return datetime(int_vec[2], int_vec[0], int_vec[1])

df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')

diagage_list, ctage_list = [], []
for i, item in df.iterrows():
    #print (len(df), i)
    
    if item['Date of Birth'] == item['Date of Birth'] and item['Diagnosis Date'] == item['Diagnosis Date']:
        diagage = convert_to_time2(item['Diagnosis Date']) - convert_to_time1(item['Date of Birth'])
        diagage = round(diagage.days / 365.0, 2)
        try:
            assert diagage < 100 and diagage > 20
            diagage_list.append(diagage)
        except:
            print (item['Date of Birth'], item['Diagnosis Date'], diagage, item['Patient ID'])
            diagage_list.append('')
    else:
        diagage_list.append('')
        
    if item['Date of Birth'] == item['Date of Birth'] and item['Chest CT Date (MM/YY)'] == item['Chest CT Date (MM/YY)']:
        try:
            ctage = convert_to_time1(item['Chest CT Date (MM/YY)']) - convert_to_time1(item['Date of Birth'])
        except:
            ctage = convert_to_time1(item['Chest CT Notes']) - convert_to_time1(item['Date of Birth'])
        ctage = round(ctage.days / 365.0, 2)
        try:
            assert ctage < 100 and ctage > 20
            ctage_list.append(ctage)
        except:
            print (item['Date of Birth'], item['Chest CT Date (MM/YY)'] , ctage, item['Patient ID'])
            ctage_list.append('')
    else:
        ctage_list.append('')
df['diagage'] =  diagage_list 
df['ctage'] = ctage_list
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

Jan 30, 1989 6/18/93 4.38 1127
Jan 26, 2015 Jul 6, 2015 0.44 11703
Dec 29, 1988 Nov 3, 2004 15.86 11772
Aug 8, 1986 Jul 7, 2005 18.93 11775
Nov 25, 1933 11/30/37 4.02 1692
May 5, 2001 10/11/01 0.44 2199
Dec 13, 1966 7/1/82 15.56 3049
Jul 26, 1977 9/1/80 3.1 3140
Jun 1, 1978 2/1/96 17.68 4044
Dec 16, 1988 7/6/00 11.56 406
Jun 10, 1945 Apr 5, 5005 3061.85 5322
Sep 17, 1992 Nov 23, 2011 19.19 8661


In [9]:
import re
def convert_to_time2(string):
    str_vec = re.split('[/]', string)
    
    assert len(str_vec) == 3
    int_vec = [int(i) for i in str_vec]
    
    assert int_vec[0] < 13 and int_vec[1] < 32 and int_vec[2] < 100

    if int_vec[2] < 21:
        int_vec[2] += 2000
    else:
        int_vec[2] += 1900
    
    
    return str(int_vec[2]) +  str(int_vec[0]).zfill(2) + str(int_vec[1]).zfill(2)


df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
diagdate_norm = [] 
for i, item in df.iterrows():
    if item['Diagnosis Date'] == item['Diagnosis Date']:
        diagdate_norm.append(convert_to_time2(str(item['Diagnosis Date'])))
    else:
        diagdate_norm.append('')
df['diagdate_norm'] = diagdate_norm
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

In [134]:
def Mayo(age, ciga, cancer, diameter, spicul, upper):
    ori_val = -6.827 + 0.0391 * age + 0.7917 * ciga +  1.3388 * cancer + 0.1274 * diameter + 1.0407 * spicul + 0.7838 * upper
    prob = np.exp(ori_val) / (1 + np.exp(ori_val))
    return round(ori_val, 3), round(prob, 3)
mayo_diag, mayo_ct =[], []
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
for i, item in df.iterrows():
    if item['prior_cancer'] != item['prior_cancer'] or item['upperlobe'] != item['upperlobe'] or item['diagage'] != item['diagage'] or item['spicul'] != item['spicul'] or item['nodule_size'] != item['nodule_size'] or item['smok'] != item['smok']:
        mayo_diag.append('')
    else:
        mayo_diag.append(Mayo(item['diagage'], item['smok'], item['prior_cancer'], item['nodule_size'], item['spicul'], item['upperlobe'])[-1])
        
    if item['prior_cancer'] != item['prior_cancer'] or item['upperlobe'] != item['upperlobe'] or item['ctage'] != item['ctage'] or item['spicul'] != item['spicul'] or item['nodule_size'] != item['nodule_size'] or item['smok'] != item['smok']:
        mayo_ct.append('')
    else:
        mayo_ct.append(Mayo(item['ctage'], item['smok'], item['prior_cancer'], item['nodule_size'], item['spicul'], item['upperlobe'])[-1])
        
df['mayo_diag']  = mayo_diag
df['mayo_ct']  = mayo_ct
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv', index = False)

In [3]:
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
df = df.query('cancer_bengin == cancer_bengin')
df = df.loc[df['withimg_0330'] == 1]
df = df.loc[(df['mayo_diag'] ==df['mayo_diag']) | (df['mayo_ct'] ==df['mayo_ct'])]
df_cancer = df.query('cancer_bengin == 1')
print (len(set(df['MCL_ID'].tolist())), len(set(df_cancer['MCL_ID'].tolist())))

716 554


In [5]:
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
df = df.query('cancer_bengin == cancer_bengin')
df = df.loc[df['withimg_0330'] == 1]
df = df.loc[(df['mayo_diag'] ==df['mayo_diag']) | (df['mayo_ct'] ==df['mayo_ct'])]

df = df.query('new_in0398 == 1 or new_in1730 == 1 or new_in0136 == 1 or new_in1078 == 1')

df = df.query('nodule_size > 5.9 and nodule_size < 30.1')

mcl_list = []
cancer_list = []
for i, item in df.iterrows():
    try:
        if item['mayo_diag'] == item['mayo_diag'] and item['mayo_diag'] >= 0.05 and item['mayo_diag'] <= 0.65:
            mcl_list.append(item['MCL_ID'])
            if item['cancer_bengin'] == 1:
                cancer_list.append(item['MCL_ID'])
        elif item['mayo_ct'] == item['mayo_ct'] and item['mayo_ct'] >= 0.05 and item['mayo_ct'] <= 0.65:
            mcl_list.append(item['MCL_ID'])
            if item['cancer_bengin'] == 1:
                cancer_list.append(item['MCL_ID'])
    except:
        print (item['mayo_diag'], item['mayo_ct'])
print (len(set(mcl_list)), len(set(cancer_list)))


357 256


In [13]:
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_part1_nodup.csv')
df = df.query('cancer_bengin == cancer_bengin')
df = df.loc[df['withimg_0330'] == 1]
df = df.loc[(df['mayo_diag'] ==df['mayo_diag']) | (df['mayo_ct'] ==df['mayo_ct'])]

df = df.query('new_in0398 == 1 or new_in1730 == 1 or new_in0136 == 1 or new_in1078 == 1')

df = df.query('nodule_size > 5.9 and nodule_size < 30.1')

df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_mayo5_65_nod6_30.csv', index = False)

In [20]:
# to see how many long in MCL with csv files
import re
df  = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Riqiang_5_12_2020_8_52_28.csv')
subj_sess = {}
for i, item in df.iterrows():
    tmp_list = re.split('[_-]', item['XNAT_CTSESSIONDATA ID'])
    subj = tmp_list[0]
    if tmp_list[0] == tmp_list[1]:
        sess = tmp_list[2]
    else:
        sess = tmp_list[1]
    if len(sess) != 8 and len(tmp_list) >=4:
        sess = tmp_list[3] + tmp_list[1] + tmp_list[2]
    try:
        assert len(sess) == 8
    except:
        print (item['XNAT_CTSESSIONDATA ID'])
        continue
    if subj not in subj_sess.keys():
        subj_sess[subj] = []
    subj_sess[subj].append(sess)



13198007362_05_02
13899683756_04_27
18880161965_03_13
18880161965_04
18880161965_04_03
18880161965_05
18880161965_08
18880161965_11_14
19669946331__093734_859000
20817757605_07_16
23797390280_04
23797390280_06
30041657825__130053_000000
31041894607__171930_606000
33400829042__101902
35016263731_04_03
3706595794_2_20181114
37660576714_2_20171003
39479816584__152913_125000
40437992060_08
42157034411_05
5594368416_1_20190128_094102
5594368416_2_20190730_081000_812000
5847948632_1_20190718_171829_437000
5847948632_2_20190927_104258_653000
7107891308_04_12


In [52]:
# add how many cts in csv file

data_root = '/nfs/masi/MCL/nifti/combine'
subj_list = os.listdir(data_root)
subj_sess = {}

for i in range(len(subj_list)):
    sess_list = os.listdir(data_root + '/' + subj_list[i])
    subj_sess[subj_list[i]] = len(sess_list)

df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_mayo5_65_nod6_30.csv')
cntCT = []
for i, item in df.iterrows():
    subj = str(item['MCL_ID']).replace('.0', '')
    if subj in subj_sess.keys():
        cntCT.append(subj_sess[subj])
    else:
        cntCT.append('')
print (cntCT)
df['cntCT'] = cntCT
df.to_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_mayo5_65_nod6_30.csv', index = False)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, '', 6, 6, 4, 4, 4, 1, 1, 1, 1, 2, 2, 6, 6, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, '', 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 5, 3, 4, 4, 4, 1, 1, 1, 1, 1, 2, 2, 2, 2, 4, 4, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, '', 1, 1, 3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 2, '', '', 1, 4, 4, 4, 4, 5, 7, 7, 7, 7, 2, 1, 1, 1, 1, 1, 1, 1, 4, 2, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 7, 7, 7, 7, 0, 0, 9, 0, 3, 1, 4, '', 1, 2, 2, 1, 1, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 3, 3, 1, 1, 5, 5, 5, 5, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 2, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 2, 1, 1, 3, 4, 4, 4, 4, 1, 1, 1, 1, 0, 4, 4, 0, 0, 0, 4, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 6, 6, 6, 0, 0, 2, 2, 

In [54]:
df = pd.read_csv('/nfs/masi/MCL/file/clinical/LabelFile/0421/Report_mayo5_65_nod6_30.csv')

df = df.query('cntCT > 1')
#df = df.query('cancer_bengin == 1')
print (len(set(df['MCL_ID'].tolist())))
        


133


In [ ]:
all_list = pd.read_csv('/nfs/masi/NLST/package-nlst-7-2018.09.24')

In [85]:
# this is a tmp cell, for ppm 

df = pd.read_excel('/nfs/masi/NLST/file/for_PPM/ICs from NLST.xlsx', sheet_name = 'Sheet1')
#df = pd.read_excel('/nfs/masi/NLST/file/for_PPM/latestage nlst.xlsx', sheet_name = 'Sheet1')
#df = pd.read_excel('')
pid = df['pid'].tolist()
pid = [str(i) for i in pid]

data_list = os.listdir('/nfs/masi/gaor2/tmp/dcm4che_push')
#data_list = 
print (set(pid) - set(data_list))
# for subj in pid:
#     print (subj)
#     os.system('cp -r /nfs/masi/NLST/ORI_DICOM/NLST_cancer/' + subj + ' /nfs/masi/gaor2/tmp/dcm4che_push/')

{'209170', '218583'}


In [84]:
data_root = '/nfs/masi/gaor2/tmp/dcm4che_push'
data_list = os.listdir(data_root)
for subj in data_list:
    #print ('/home/local/VANDERBILT/gaor2/dcm4che-5.22.2-bin/dcm4che-5.22.2/bin/storescu -c HMUVMCSCP@healthmyne-dev.app.vumc.org:1040 ' + data_root + '/' + subj)
    #os.system('/home/local/VANDERBILT/gaor2/dcm4che-5.22.2-bin/dcm4che-5.22.2/bin/storescu -c HMUVMCSCP@healthmyne-dev.app.vumc.org:1040 ' + data_root + '/' + subj)

SyntaxError: unexpected EOF while parsing (<ipython-input-84-d5b6492536cd>, line 5)

17628


In [ ]:
''